In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
path = r'hyokorevBM_jgd2024_h.par'
df = pd.read_csv(path, skiprows=15, sep="\\s+", encoding='Shift_JIS')

 - メッシュコード：https://club.informatix.co.jp/?p=1226

In [3]:
def meshcode_to_center_latlon(meshcode):
    meshcode = int(meshcode)
    p = meshcode // 10**6
    q = (meshcode // 10**4) % 100
    r = (meshcode // 10**3) % 10
    s = (meshcode // 10**2) % 10
    t = (meshcode // 10) % 10
    u = meshcode % 10

    lat_unit = 2 / 3  # degrees
    lon_unit = 1.0

    lat = p * lat_unit + r * (lat_unit / 8) + t * (lat_unit / 8 / 10)
    lon = (q + 100) * lon_unit + s * (lon_unit / 8) + u * (lon_unit / 8 / 10)

    return lat, lon

df[["lat", "lon"]] = df["MeshCode"].apply(
    lambda x: pd.Series(meshcode_to_center_latlon(x))
)

In [4]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:6668")

In [5]:
gdf.drop(columns=["MeshCode","0.00000","lat", "lon"], inplace=True)
gdf.to_parquet("hyokorevBM_jgd2024_h.parquet", compression="zstd")